# Business Inteligence Project
### Sebastián Huneeus

#### sebastian@codept.de
#### [github page](https://shuneeus.github.io/)


Using actual data from Facebook Campaigns from an App, I developed a business analysis to find which campaigns were more effective. For doing so, I pulled data from a server, joined the tables using  SQL queries and calculated summary statistics and visualizations. Comparing the performance of web and app campaigns across multiple regions, I came up with some insights and recommendations for the marketing management team. 

The main actions taken were: 

* 1 Pulling the data from the server and a first sanity check. 
* 2 KPI definition and calculation. 
* 3 Visualizations, insights and recommendations to the head of the marketing team.

# 1. Data retrieving 

In [1]:
import psycopg2
import pandas as pd
import sidetable
import nbconvert

In [ ]:
Connection to SQL databases using Pyscopg

In [2]:
conn = psycopg2.connect(
    host="d-an-test.cygqmbt4gi1l.eu-central-1.rds.amazonaws.com",
    database="postgres",
    user="sebastian_huneeus",
    password= ***********)

# 1.1 Data sanity checks
### Checking for duplicates & missing in the tables.


### Dowloading Table1: fbt_web_sessions

In [ ]:
fbt_web_sessions = "select * from fbt_web_sessions"

fbt_web_sessions_df = pd.read_sql(fbt_web_sessions, conn)

In [3]:
fbt_web_sessions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447858 entries, 0 to 447857
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   sk_web_user                447858 non-null  int64 
 1   session_id                 447858 non-null  object
 2   session_started_timestamp  447858 non-null  object
 3   network                    447858 non-null  object
 4   campaign_id                447858 non-null  int64 
 5   sub_region                 447858 non-null  object
dtypes: int64(2), object(4)
memory usage: 23.9+ MB


In [4]:
fbt_web_sessions_df.describe()  

,sk_web_user,campaign_id
count,4.478580e+05,447858.0
mean,7.635981e+06,280020.0
std,6.797583e+05,0.0
min,4.088400e+04,280020.0
25%,7.066468e+06,280020.0
50%,7.632792e+06,280020.0
75%,8.152908e+06,280020.0
max,1.220544e+07,280020.0


In [5]:
fbt_web_sessions_df.pivot_table(index = ['sk_web_user'], aggfunc ='size') 

sk_web_user
40884       1
684986      1
1107846     1
1363669     1
1482496     1
           ..
12178796    1
12181610    1
12191672    1
12192443    1
12205439    1
Length: 447858, dtype: int64

### Dowloading of Table2: fbt_web_user

In [ ]:
fbt_web_user = "select * from fbt_web_user_mapping"

fbt_web_user_df = pd.read_sql(fbt_web_user, conn)

In [6]:
fbt_web_user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159537 entries, 0 to 159536
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   sk_user        159537 non-null  int64 
 1   domain_userid  159537 non-null  object
 2   sk_web_user    159537 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.9+ MB


In [8]:
fbt_web_user_df.pivot_table(index = ['sk_web_user'], aggfunc ='size') # a very small fraction of duplicates 

sk_web_user
1107846     1
1868613     1
2035957     1
2359779     1
2422795     1
           ..
11940763    1
12157745    1
12170619    1
12181610    1
12205439    1
Length: 158630, dtype: int64

### Dowloading of Table3: fbt_facebook_campaigns



In [9]:
fbt_campaigns = "select * from fbt_facebook_campaigns"

fbt_campaigns_df = pd.read_sql(fbt_campaigns, conn)

fbt_campaigns_df

,campaign_name,campaign_id,spend_date,sub_region,impressions,spend
0,web_campaign_world,280020,2020-11-27,Northern America,82722,3235.0
1,web_campaign_world,280020,2020-11-27,Northern & Western Europe,78332,1558.0
2,ios_campaign_world,279755,2020-11-27,Southern & Eastern Europe,48815,99.0
3,web_campaign_world,280020,2020-11-27,Southern & Eastern Europe,136204,1086.0
4,ios_campaign_world,279755,2020-11-27,Northern America,19382,271.0
...,...,...,...,...,...,...
412,web_campaign_world,280020,2020-11-28,Northern America,127710,5021.0
413,android_campaign_world,339703,2020-11-27,Australasia & Asia,240990,29.0
414,android_campaign_world,339703,2020-11-27,Northern & Western Europe,23532,132.0
415,ios_campaign_north_america,379581,2020-11-27,Northern America,38841,718.0






# 1.2 Exploratory analysis

The following table shows that there are four campaigns in the data. The count and percentages represent the number of times that each campaign appears in the data set. 

In [10]:
fbt_campaigns_df.stb.freq(["campaign_id"])

,campaign_id,count,percent,cumulative_count,cumulative_percent
0,339703,150,35.971223,150,35.971223
1,279755,150,35.971223,300,71.942446
2,280020,90,21.582734,390,93.525180
3,379581,27,6.474820,417,100.000000


### Campaign spenditure 

The expenditure amount differs vastly among campaigns.  Campaign ID 279755 sets the superior spending cap ($155,666). However, the figures don't allow us to tell which campaigns are performing better and worse.

In [12]:
add_spend_by_campaign = "select \
                        campaign_id, \
                        sum(spend) spend\
                        from \
                        fbt_facebook_campaigns \
                        group by \
                        campaign_id"

spend_by_campaign_df = pd.read_sql(add_spend_by_campaign, conn)
spend_by_campaign_df

,campaign_id,spend
0,339703,46391.0
1,379581,63239.0
2,280020,68982.0
3,279755,155666.0


### Having concluded with the sanity check and exploratory analysis, I  conclude that: 
   * there are four campaigns, with different expenditure volume 
   * there are no substantial amount of duplicates
   * there are no substantial amount of missing values
   
The data is largely robust, and we can proceed to build a more informative analysis. 

# 2. KPI: Return on ad-spend (ROAS)

The data allows calculating the return on ads-spending KPI (ROAS). This KPI represents the fraction of conversions due to ad exposure divided by the expenditure amount of the campaign. The metric represents the effectiveness of marketing campaigns. Some marketing managers set the ROAS threshold in 0.9; a ROAS of superior to 0.9 is considered an acceptable return on the investment.

In the next section, I'll pull the data from the different tables, merge them and calculate the ROAS for web and app Facebook campaigns. To increase the granularity of the analysis, I nested the measurements into regional clusters.   

### 2.1. Web ROAS

In [ ]:
web_conversions_df= "select\
                        c.sub_region, \
                        e.campaign_id,\
                        sum(a.revenue) revenue,\
                        g.spend spend \
                    from \
                        fbt_conversions_web a,\
                        fbt_web_user_mapping b,\
                        fbt_web_sessions c,\
                            (select   \
                             distinct d.campaign_id\
                             from fbt_facebook_campaigns d) e,\
                             (select   \
                                sub_region, campaign_id, sum(spend) spend\
                             from fbt_facebook_campaigns f\
                             group by \
                                 sub_region, \
                                 campaign_id) g\
                    where\
                        a.sk_user = b.sk_user\
                    and \
                        b.sk_web_user = c.sk_web_user \
                    and \
                        c.campaign_id = e.campaign_id \
                    and \
                        c.sub_region = g.sub_region \
                    and       \
                        c.campaign_id = g.campaign_id\
                    group by\
                        e.campaign_id, \
                        c.sub_region,\
                        g.spend "
                        
web_conversionsdf = pd.read_sql(web_conversions_df, conn)
web_conversionsdf["WEB_ROAS"]=web_conversionsdf["revenue"]/web_conversionsdf["spend"]
web_conversionsdf.round(2)
web_conversionsdf.to_csv("outputs/web_roas.cvs")

In [5]:
web_roas = pd.read_csv("outputs/web_roas.cvs")

In [17]:
web_roas.sort_values(by=['WEB_ROAS']).round(2)

,Unnamed: 0,sub_region,campaign_id,revenue,spend,WEB_ROAS
2,2,Northern & Western Europe,280020,10220.16,16404.0,0.62
4,4,Southern & Eastern Europe,280020,8425.15,11906.0,0.71
1,1,Northern America,280020,20774.48,28836.0,0.72
3,3,South & Central America,280020,4218.88,4623.0,0.91
0,0,Australasia & Asia,280020,7359.31,7213.0,1.02


## 2.2. App ROAS

In [ ]:
 app_revenue_cost_by_camp_reg=    "select \
                                    f.sub_region, \
                                    f.campaign_id,\
                                    sum(a.revenue) app_revenue, \
                                    f.spend spend\
                                from \
                                    fbt_conversions_app a, \
                                    fbt_installs b,\
                                        (select  \
                                         distinct c.campaign_id\
                                         from fbt_facebook_campaigns c) d,\
                                         (select   \
                                            sub_region, campaign_id, sum(spend) spend\
                                         from fbt_facebook_campaigns\
                                         group by \
                                             sub_region, \
                                             campaign_id) f\
                                where \
                                    a.sk_user = b.sk_user \
                                and \
                                    b.campaign_id = d.campaign_id\
                                and \
                                    b.campaign_id =f.campaign_id\
                                and \
                                    b.sub_region = f.sub_region\
                                group by \
                                    f.campaign_id, \
                                    f.sub_region,\
                                    f.spend"

app_conv_rev_cost_by_reg_camp = pd.read_sql(app_revenue_cost_by_camp_reg, conn)
app_conv_rev_cost_by_reg_camp["APP_ROAS"]=app_conv_rev_cost_by_reg_camp["app_revenue"]/app_conv_rev_cost_by_reg_camp["spend"]
app_conv_rev_cost_by_reg_camp.round(2)
app_conv_rev_cost_by_reg_camp.to_csv("outputs/app_roas.cvs")

In [13]:
app_roas = pd.read_csv("outputs/app_roas.cvs")
app_roas.sort_values(by=['APP_ROAS']).round(2)

,Unnamed: 0,sub_region,campaign_id,app_revenue,spend,APP_ROAS
10,10,Northern America,379581,23684.52,63239.0,0.37
9,9,Southern & Eastern Europe,339703,4331.14,5608.0,0.77
1,1,Northern America,279755,40208.91,49692.0,0.81
3,3,South & Central America,279755,19061.40,23106.0,0.82
7,7,Northern & Western Europe,339703,21641.83,23544.0,0.92
8,8,South & Central America,339703,5626.17,6107.0,0.92
4,4,Southern & Eastern Europe,279755,17029.55,17913.0,0.95
5,5,Australasia & Asia,339703,2541.79,2488.0,1.02
0,0,Australasia & Asia,279755,13213.37,12651.0,1.04
6,6,Northern America,339703,10073.74,8644.0,1.17
